In [97]:
import pandas as pd

In [98]:
directory = 'drive/MyDrive/Data'
cohort_name = 'USA'

In [99]:
annovar_path = f'{directory}/{cohort_name}_withgnomad_clinvar.annovar.hg38_multianno.txt'
annovar_df = pd.read_csv(annovar_path, sep='\t')
columns_to_extract = [
    'Chr', 'Start', 'End', 'Ref', 'Alt', 'Func.refGene',
    'GeneDetail.refGene', 'ExonicFunc.refGene', 'AAChange.refGene',
    'CADD_phred', 'CLNSIG', 'gnomad41_genome_AF',
    'gnomad41_genome_AF_asj', 'gnomad41_genome_AF_nfe','avsnp151']

# Extract the relevant columns
annovar_df = annovar_df[columns_to_extract]

# Display the first few rows of the DataFrame
annovar_df

,Chr,Start,End,Ref,Alt,Func.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,CADD_phred,CLNSIG,gnomad41_genome_AF,gnomad41_genome_AF_asj,gnomad41_genome_AF_nfe,avsnp151
0,12,40225280,40225280,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon1:c.G149A:p.R50H,1.908,Benign,0.9720,1.0000,0.9998,rs2256408
1,12,40225580,40225580,T,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon2:c.T177A:p.N59K,15.63,Conflicting_classifications_of_pathogenicity,9.855e-05,0.0014,7.349e-05,rs150422099
2,12,40225628,40225628,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon2:c.G225A:p.A75A,.,Benign/Likely_benign,0.0015,0,0,rs75054132
3,12,40225694,40225694,G,A,intronic,.,.,.,.,.,0.0011,0.0014,0.0020,rs367949688
4,12,40225725,40225725,T,C,intronic,.,.,.,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,12,40368579,40368579,T,G,UTR3,NM_198578:c.*814T>G,.,.,.,.,.,.,.,.
139,12,40368834,40368834,A,G,UTR3,NM_198578:c.*1069A>G,.,.,.,Uncertain_significance,0.0003,0,0.0003,rs199823070
140,12,40368876,40368876,G,A,UTR3,NM_198578:c.*1111G>A,.,.,.,.,1.98e-05,0.0006,0,rs948032392
141,12,40368946,40368946,C,T,UTR3,NM_198578:c.*1181C>T,.,.,.,Benign,0.0027,0,0,rs147956597


In [100]:
# Define the path to the freq and counts files
freq_path = f'{directory}/{cohort_name}_MAF.assoc.fisher'
counts_path = f'{directory}/{cohort_name}.assoc.fisher'
freq_df = pd.read_csv(freq_path, sep='\s+')
counts_df = pd.read_csv(counts_path, sep='\s+')

# Format data types to match
freq_df['SNP'] = freq_df['SNP'].astype(str)
counts_df['SNP'] = counts_df['SNP'].astype(str)

# merge the two files
MAF_and_counts_df = pd.merge(counts_df[['CHR', 'SNP', 'BP', 'A1', 'A2', 'C_A', 'C_U']],
                             freq_df[['SNP', 'F_A', 'F_U']],
                             on='SNP')

# Display the first few rows of the merged DataFrame
MAF_and_counts_df.head()


,CHR,SNP,BP,A1,A2,C_A,C_U,F_A,F_U
0,12,rs2256408:G:A,40225280,G,A,4,5,0.001646,0.004621
1,12,rs150422099:T:A,40225580,A,T,1,0,0.000402,0.000000
2,12,rs75054132:G:A,40225628,A,G,0,1,0.000000,0.000906
3,12,rs367949688:G:A,40225694,A,G,2,1,0.000797,0.000888
4,12,12:40619527:T:C,40225725,C,T,10,7,0.004026,0.006306


In [101]:
# Count number of rows
print(f'Number of rows: {len(MAF_and_counts_df)}')

Number of rows: 143


In [102]:
# Based on CHR, A1 and A2 columns, merge the annovar and MAF_and_counts dataframes
MAF_and_counts_df['CHR'] = MAF_and_counts_df['CHR'].astype(str)
MAF_and_counts_df['A1'] = MAF_and_counts_df['A1'].astype(str)
MAF_and_counts_df['A2'] = MAF_and_counts_df['A2'].astype(str)
annovar_df['Chr'] = annovar_df['Chr'].astype(str)
if cohort_name=='UKBB':
  # remove Chr from 'Chr'
  annovar_df['Chr'] = annovar_df['Chr'].str.replace('chr', '')
annovar_df['Ref'] = annovar_df['Ref'].astype(str)
annovar_df['Alt'] = annovar_df['Alt'].astype(str)
merged_df = pd.merge(annovar_df, MAF_and_counts_df, left_on=['Chr', 'Start', 'Ref', 'Alt'], right_on=['CHR', 'BP', 'A2', 'A1'])
merged_df = merged_df.drop(columns=['CHR', 'BP', 'A1', 'A2'])
# Display the first few rows of the merged DataFrame
merged_df.head()

,Chr,Start,End,Ref,Alt,Func.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,CADD_phred,CLNSIG,gnomad41_genome_AF,gnomad41_genome_AF_asj,gnomad41_genome_AF_nfe,avsnp151,SNP,C_A,C_U,F_A,F_U
0,12,40225580,40225580,T,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon2:c.T177A:p.N59K,15.63,Conflicting_classifications_of_pathogenicity,9.855e-05,0.0014,7.349e-05,rs150422099,rs150422099:T:A,1,0,0.000402,0.000000
1,12,40225628,40225628,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon2:c.G225A:p.A75A,.,Benign/Likely_benign,0.0015,0,0,rs75054132,rs75054132:G:A,0,1,0.000000,0.000906
2,12,40225694,40225694,G,A,intronic,.,.,.,.,.,0.0011,0.0014,0.0020,rs367949688,rs367949688:G:A,2,1,0.000797,0.000888
3,12,40225725,40225725,T,C,intronic,.,.,.,.,.,.,.,.,.,12:40619527:T:C,10,7,0.004026,0.006306
4,12,40232255,40232255,A,G,intronic,.,.,.,.,.,6.566e-06,0,0,rs540841916,rs540841916:A:G,0,0,0.000000,0.000000


In [103]:
# Add domain annotation based on BP location
domain_df = pd.read_excel(f'{directory}/LRRK2_GRCh38.xlsx', sheet_name="Sheet2")

domain_df.head()

,Chromosome,Gene,Domain,Column1,Column12,narrow_start,narrow_end,wide_start,wide_end
0,12,LRRK2,ARM,100-688,100-688,40225234,40274891,40225153,40277914
1,12,LRRK2,ANK,688-863,688-863,40277916,40284040,40277915,40284043
2,12,LRRK2,LRR,943-1309,943-1309,40294854,40305881,40294854,40305881
3,12,LRRK2,RocCOR,1327-1842,1327-1842,40309151,40313983,40308486,40322084
4,12,LRRK2,Kinase,1879-2138,1879-2138,40323303,40340349,40323249,40351571


In [104]:
def annotate_domain(row, domain_df):
    for _, domain_row in domain_df.iterrows():
        if domain_row['wide_start'] <= row['Start'] <= domain_row['wide_end']:
            return domain_row['Domain']
    return 'Unknown'

# Annotate domain information of variants
merged_df['Domain'] = merged_df.apply(annotate_domain, axis=1, domain_df=domain_df)

# Display the first few rows of the merged DataFrame
merged_df.head()
# Count each domain group
domain_counts = merged_df['Domain'].value_counts()
print(domain_counts)

Domain
ARM        35
Unknown    25
RocCOR     21
Kinase     16
WD40       15
LRR        13
ANK         6
Name: count, dtype: int64


In [105]:
# Count number of rows
print(f'Number of rows: {len(merged_df)}')

# Show me the unmatched rows from MAF_and_counts_df
unmatched_rows = MAF_and_counts_df[~MAF_and_counts_df['SNP'].isin(merged_df['SNP'])]
unmatched_rows.head()
# This is before the UTR5, intronic, and UTR3 variants are removed

Number of rows: 131


,CHR,SNP,BP,A1,A2,C_A,C_U,F_A,F_U
0,12,rs2256408:G:A,40225280,G,A,4,5,0.001646,0.004621
5,12,12:40619531:TG:T,40225729,T,TG,9,3,0.003603,0.002683
6,12,12:40619533:GT:G,40225731,G,GT,0,0,0.000000,0.000000
7,12,12:40619536:AT:A,40225734,A,AT,4,1,0.001595,0.000888
8,12,12:40619538:TC:T,40225736,T,TC,3,0,0.001209,0.000000


In [106]:
# Remove UTR5, intronic, and UTR3 variants and recount unmatched
merged_df = merged_df[~merged_df['Func.refGene'].isin(['UTR5', 'intronic', 'UTR3'])]
print(f'Number of rows: {len(merged_df)}')

# Count again
unmatched_rows = MAF_and_counts_df[~MAF_and_counts_df['SNP'].isin(merged_df['SNP'])]
unmatched_rows.head()
# This is after the UTR5, intronic, and UTR3 variants are removed
# Count each domain group
domain_counts = merged_df['Domain'].value_counts()
print(domain_counts)

Number of rows: 79
Domain
ARM        20
RocCOR     16
Unknown    11
Kinase     10
WD40       10
LRR         7
ANK         5
Name: count, dtype: int64


In [107]:
# remove Domain == Unknown
merged_df = merged_df[merged_df['Domain'] != 'Unknown']
print(f'Number of rows: {len(merged_df)}')

Number of rows: 68


In [108]:
# Make the headers more legible
# Mapping of old headers to new headers without underscores
header_mapping = {
    'Chr': 'Chromosome',
    'Start': 'Start Position',
    'End': 'End Position',
    'Ref': 'Reference Allele',
    'Alt': 'Alternate Allele',
    'Func.refGene': 'Functional Annotation',
    'GeneDetail.refGene': 'outside gene cDNA Change',
    'ExonicFunc.refGene': 'Exonic Function',
    'AAChange.refGene': 'cDNA and Amino Acid Change',
    'CADD_phred': 'CADD v1.7 Phred Score',
    'CLNSIG': 'Clinical Significance (Varsome)',
    'gnomad41_genome_AF': 'gnomAD Allele Frequency',
    'gnomad41_genome_AF_asj': 'gnomAD AF Ashkenazi',
    'gnomad41_genome_AF_nfe': 'gnomAD AF Non Finnish European',
    'avsnp151': 'rsID'
}
merged_df = merged_df.rename(columns=header_mapping)

# Display the first few rows of the merged DataFrame
merged_df.head()

,Chromosome,Start Position,End Position,Reference Allele,Alternate Allele,Functional Annotation,outside gene cDNA Change,Exonic Function,cDNA and Amino Acid Change,CADD v1.7 Phred Score,...,gnomAD Allele Frequency,gnomAD AF Ashkenazi,gnomAD AF Non Finnish European,rsID,SNP,C_A,C_U,F_A,F_U,Domain
0,12,40225580,40225580,T,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon2:c.T177A:p.N59K,15.63,...,9.855e-05,0.0014,7.349e-05,rs150422099,rs150422099:T:A,1,0,0.000402,0.000000,ARM
1,12,40225628,40225628,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon2:c.G225A:p.A75A,.,...,0.0015,0,0,rs75054132,rs75054132:G:A,0,1,0.000000,0.000906,ARM
5,12,40232380,40232380,A,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon3:c.A344C:p.H115P,17.96,...,0.0001,0.0009,2.94e-05,rs201439315,rs201439315:A:C,2,0,0.000797,0.000000,ARM
6,12,40235634,40235634,T,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon4:c.T356C:p.L119P,19.51,...,0.0014,0,0.0023,rs33995463,rs33995463:T:C,2,2,0.000797,0.001776,ARM
7,12,40235642,40235642,C,T,exonic,.,synonymous SNV,LRRK2:NM_198578:exon4:c.C364T:p.L122L,.,...,0.0003,0.0003,0.0005,rs41286468,rs41286468:C:T,2,1,0.000797,0.000888,ARM


In [109]:
header_mapping_2 = {
    'C_A': 'N (Case)',
    'C_U': 'N (Control)',
    'F_A': 'MAF (Case)',
    'F_U': 'MAF (Control)'
}

merged_df = merged_df.rename(columns=header_mapping_2)
merged_df

,Chromosome,Start Position,End Position,Reference Allele,Alternate Allele,Functional Annotation,outside gene cDNA Change,Exonic Function,cDNA and Amino Acid Change,CADD v1.7 Phred Score,...,gnomAD Allele Frequency,gnomAD AF Ashkenazi,gnomAD AF Non Finnish European,rsID,SNP,N (Case),N (Control),MAF (Case),MAF (Control),Domain
0,12,40225580,40225580,T,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon2:c.T177A:p.N59K,15.63,...,9.855e-05,0.0014,7.349e-05,rs150422099,rs150422099:T:A,1,0,0.000402,0.000000,ARM
1,12,40225628,40225628,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon2:c.G225A:p.A75A,.,...,0.0015,0,0,rs75054132,rs75054132:G:A,0,1,0.000000,0.000906,ARM
5,12,40232380,40232380,A,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon3:c.A344C:p.H115P,17.96,...,0.0001,0.0009,2.94e-05,rs201439315,rs201439315:A:C,2,0,0.000797,0.000000,ARM
6,12,40235634,40235634,T,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon4:c.T356C:p.L119P,19.51,...,0.0014,0,0.0023,rs33995463,rs33995463:T:C,2,2,0.000797,0.001776,ARM
7,12,40235642,40235642,C,T,exonic,.,synonymous SNV,LRRK2:NM_198578:exon4:c.C364T:p.L122L,.,...,0.0003,0.0003,0.0005,rs41286468,rs41286468:C:T,2,1,0.000797,0.000888,ARM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,12,40363462,40363462,C,T,exonic,.,synonymous SNV,LRRK2:NM_198578:exon48:c.C7089T:p.L2363L,.,...,0.0002,0,0,rs201329694,rs201329694:C:T,1,0,0.000398,0.000000,WD40
116,12,40363526,40363526,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon48:c.G7153A:p.G2385R,11.64,...,0.0008,0,0,rs34778348,rs34778348:G:A,0,2,0.000000,0.001776,WD40
118,12,40364842,40364842,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon49:c.G7182A:p.R2394R,.,...,.,.,.,.,12:40758644:G:A,1,0,0.000398,0.000000,WD40
119,12,40364997,40364997,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon49:c.G7337A:p.R2446H,13.42,...,6.583e-06,0,1.473e-05,rs200795955,rs200795955:G:A,0,1,0.000000,0.000911,WD40


In [110]:
# Now we organize the cDNA and Amino Acid Change column and separate into cDNA and AA Change separate columns
for index, row in merged_df.iterrows():
    if pd.notna(row['cDNA and Amino Acid Change']):
        split_values = row['cDNA and Amino Acid Change'].split(':')[-2:]
        merged_df.at[index, 'cDNA'] = split_values[0]
        merged_df.at[index, 'Amino Acid Change'] = split_values[1] if len(split_values) > 1 else None

merged_df

,Chromosome,Start Position,End Position,Reference Allele,Alternate Allele,Functional Annotation,outside gene cDNA Change,Exonic Function,cDNA and Amino Acid Change,CADD v1.7 Phred Score,...,gnomAD AF Non Finnish European,rsID,SNP,N (Case),N (Control),MAF (Case),MAF (Control),Domain,cDNA,Amino Acid Change
0,12,40225580,40225580,T,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon2:c.T177A:p.N59K,15.63,...,7.349e-05,rs150422099,rs150422099:T:A,1,0,0.000402,0.000000,ARM,c.T177A,p.N59K
1,12,40225628,40225628,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon2:c.G225A:p.A75A,.,...,0,rs75054132,rs75054132:G:A,0,1,0.000000,0.000906,ARM,c.G225A,p.A75A
5,12,40232380,40232380,A,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon3:c.A344C:p.H115P,17.96,...,2.94e-05,rs201439315,rs201439315:A:C,2,0,0.000797,0.000000,ARM,c.A344C,p.H115P
6,12,40235634,40235634,T,C,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon4:c.T356C:p.L119P,19.51,...,0.0023,rs33995463,rs33995463:T:C,2,2,0.000797,0.001776,ARM,c.T356C,p.L119P
7,12,40235642,40235642,C,T,exonic,.,synonymous SNV,LRRK2:NM_198578:exon4:c.C364T:p.L122L,.,...,0.0005,rs41286468,rs41286468:C:T,2,1,0.000797,0.000888,ARM,c.C364T,p.L122L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,12,40363462,40363462,C,T,exonic,.,synonymous SNV,LRRK2:NM_198578:exon48:c.C7089T:p.L2363L,.,...,0,rs201329694,rs201329694:C:T,1,0,0.000398,0.000000,WD40,c.C7089T,p.L2363L
116,12,40363526,40363526,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon48:c.G7153A:p.G2385R,11.64,...,0,rs34778348,rs34778348:G:A,0,2,0.000000,0.001776,WD40,c.G7153A,p.G2385R
118,12,40364842,40364842,G,A,exonic,.,synonymous SNV,LRRK2:NM_198578:exon49:c.G7182A:p.R2394R,.,...,.,.,12:40758644:G:A,1,0,0.000398,0.000000,WD40,c.G7182A,p.R2394R
119,12,40364997,40364997,G,A,exonic,.,nonsynonymous SNV,LRRK2:NM_198578:exon49:c.G7337A:p.R2446H,13.42,...,1.473e-05,rs200795955,rs200795955:G:A,0,1,0.000000,0.000911,WD40,c.G7337A,p.R2446H


In [111]:
# Remove variants where both N (Case) and N (Control) are 0

#Count the excluded
excluded_df = merged_df[(merged_df['N (Case)'] == 0) & (merged_df['N (Control)'] == 0)]
print(f'Number of excluded rows: {len(excluded_df)}')


merged_df = merged_df[~((merged_df['N (Case)'] == 0) & (merged_df['N (Control)'] == 0))]


Number of excluded rows: 3


In [112]:
# print column names
print(merged_df.columns)

Index(['Chromosome', 'Start Position', 'End Position', 'Reference Allele',
       'Alternate Allele', 'Functional Annotation', 'outside gene cDNA Change',
       'Exonic Function', 'cDNA and Amino Acid Change',
       'CADD v1.7 Phred Score', 'Clinical Significance (Varsome)',
       'gnomAD Allele Frequency', 'gnomAD AF Ashkenazi',
       'gnomAD AF Non Finnish European', 'rsID', 'SNP', 'N (Case)',
       'N (Control)', 'MAF (Case)', 'MAF (Control)', 'Domain', 'cDNA',
       'Amino Acid Change'],
      dtype='object')


In [113]:
# if the "outside gene cDNA Change" column is not empty then split by ":" then pick third element and store it inside the cDNA of that row
for index, row in merged_df.iterrows():
    if pd.notna(row['outside gene cDNA Change']):
        split_values = row['outside gene cDNA Change'].split(':')
        if len(split_values) >= 3:
            merged_df.at[index, 'cDNA'] = split_values[2]
            # print cDNA at index
            print(merged_df.at[index, 'cDNA'])

In [114]:
# reorder columns such that
# Domain	Chromosome	Position	Reference Allele	Alternate Allele	cDNA	Amino Acid Change	Functional Annotation	Exonic Function	N (Case)	N (Control)	MAF (Case)	MAF (Control)	CADD Phred Score	Clinical Significance (Varsome)	gnomAD Allele Frequency	gnomAD AF Ashkenazi	gnomAD AF Non Finnish European
ordered_merged_df = merged_df[[
    'Domain', 'Chromosome', 'Start Position', 'Reference Allele',
    'Alternate Allele', 'cDNA', 'Amino Acid Change','rsID', 'Functional Annotation',
    'Exonic Function', 'N (Case)', 'N (Control)', 'MAF (Case)',
    'MAF (Control)', 'CADD v1.7 Phred Score', 'Clinical Significance (Varsome)',
    'gnomAD Allele Frequency', 'gnomAD AF Ashkenazi',
    'gnomAD AF Non Finnish European'
]]

ordered_merged_df.head()

,Domain,Chromosome,Start Position,Reference Allele,Alternate Allele,cDNA,Amino Acid Change,rsID,Functional Annotation,Exonic Function,N (Case),N (Control),MAF (Case),MAF (Control),CADD v1.7 Phred Score,Clinical Significance (Varsome),gnomAD Allele Frequency,gnomAD AF Ashkenazi,gnomAD AF Non Finnish European
0,ARM,12,40225580,T,A,c.T177A,p.N59K,rs150422099,exonic,nonsynonymous SNV,1,0,0.000402,0.000000,15.63,Conflicting_classifications_of_pathogenicity,9.855e-05,0.0014,7.349e-05
1,ARM,12,40225628,G,A,c.G225A,p.A75A,rs75054132,exonic,synonymous SNV,0,1,0.000000,0.000906,.,Benign/Likely_benign,0.0015,0,0
5,ARM,12,40232380,A,C,c.A344C,p.H115P,rs201439315,exonic,nonsynonymous SNV,2,0,0.000797,0.000000,17.96,Benign/Likely_benign,0.0001,0.0009,2.94e-05
6,ARM,12,40235634,T,C,c.T356C,p.L119P,rs33995463,exonic,nonsynonymous SNV,2,2,0.000797,0.001776,19.51,Conflicting_classifications_of_pathogenicity,0.0014,0,0.0023
7,ARM,12,40235642,C,T,c.C364T,p.L122L,rs41286468,exonic,synonymous SNV,2,1,0.000797,0.000888,.,Benign/Likely_benign,0.0003,0.0003,0.0005


In [115]:
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.meta_analysis import combine_effects

In [116]:
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from scipy.stats import norm



cohort_totals = {
    "USA": {"cases": 1251, "controls": 559},
    "ISRAEL": {"cases": 1091, "controls": 528},
    "FRENCH": {"cases": 1282, "controls": 2358},
    "RUSSIA": {"cases": 485, "controls": 442},
    "UKBB": {"cases": 2848, "controls": 62463},
    "AMP_PD": {"cases": 1931, "controls": 3062}
}

def calculate_confidence_interval(a, b, c, d):
    """
    Calculate 95% CI for the Odds Ratio using the normal approximation.
    a = case_count
    b = case_total - case_count
    c = control_count
    d = control_total - control_count
    """
    # Calculate OR
    if a == 0 or b == 0 or c == 0 or d == 0:
        # If any cell is zero, directly return NaNs for CI
        return np.nan, np.nan

    or_val = (a * d) / (b * c)
    # Calculate standard error of ln(OR)
    se = np.sqrt((1/a) + (1/b) + (1/c) + (1/d))
    # CI on log scale
    log_or = np.log(or_val)
    ci_lower = np.exp(log_or - 1.96 * se)
    ci_upper = np.exp(log_or + 1.96 * se)
    return ci_lower, ci_upper

def calculate_p_value_from_or(a, b, c, d):
    """
    Calculate p-value based on the Odds Ratio and its standard error.
    """
    if a == 0 or b == 0 or c == 0 or d == 0:
        return np.nan

    # Calculate Odds Ratio
    or_val = (a * d) / (b * c)

    # Calculate standard error of log(OR)
    se_log_or = np.sqrt((1/a) + (1/b) + (1/c) + (1/d))

    # Calculate z-score
    z = np.log(or_val) / se_log_or

    # Calculate two-tailed p-value
    p_value = 2 * (1 - norm.cdf(abs(z)))
    return p_value

def calculate_odds_ratio_no_correction(data, cohort_name):
    """
    Calculate Odds Ratios without continuity correction for a specific cohort.
    Use OR-based p-value calculation for consistency with OR and CI.
    """
    case_total = cohort_totals[cohort_name]["cases"]
    control_total = cohort_totals[cohort_name]["controls"]

    odds_ratios = []
    p_values = []
    ci_lowers = []
    ci_uppers = []

    for _, row in data.iterrows():
        case_count = row['N (Case)']
        control_count = row['N (Control)']

        a = case_count
        b = case_total - case_count
        c = control_count
        d = control_total - control_count

        try:
            # Calculate OR
            if a == 0 or b == 0 or c == 0 or d == 0:
                odds_ratio = np.nan
                p = np.nan
                ci_lower, ci_upper = np.nan, np.nan
            else:
                odds_ratio = (a * d) / (b * c)

                # Use OR-based p-value calculation
                p = calculate_p_value_from_or(a, b, c, d)

                # Calculate CI
                ci_lower, ci_upper = calculate_confidence_interval(a, b, c, d)

        except ZeroDivisionError:
            odds_ratio, p = np.nan, np.nan
            ci_lower, ci_upper = np.nan, np.nan

        odds_ratios.append(odds_ratio)
        p_values.append(p)
        ci_lowers.append(ci_lower)
        ci_uppers.append(ci_upper)

    data['OR'] = odds_ratios
    data['P-value'] = p_values
    data['OR 95% CI Lower'] = ci_lowers
    data['OR 95% CI Upper'] = ci_uppers
    return data


def calculate_odds_ratio_ratio_correction(data, cohort_name):
    """
    Calculate Odds Ratios with ratio-based continuity correction for a specific cohort.
    Use OR-based p-value calculation for consistency with OR and CI.
    """
    case_total = cohort_totals[cohort_name]["cases"]
    control_total = cohort_totals[cohort_name]["controls"]

    odds_ratios = []
    p_values = []
    ci_lowers = []
    ci_uppers = []

    for _, row in data.iterrows():
        case_count = row['N (Case)']
        control_count = row['N (Control)']

        # Apply ratio-based continuity correction
        a = case_count + 0.5
        b = (case_total - case_count) + 0.5
        c = control_count + 0.5
        d = (control_total - control_count) + 0.5

        try:
            # Calculate OR
            if a == 0 or b == 0 or c == 0 or d == 0:
                odds_ratio = np.nan
                p = np.nan
                ci_lower, ci_upper = np.nan, np.nan
            else:
                odds_ratio = (a * d) / (b * c)

                # Use OR-based p-value calculation
                p = calculate_p_value_from_or(a, b, c, d)

                # Calculate CI using corrected counts
                ci_lower, ci_upper = calculate_confidence_interval(a, b, c, d)

        except ZeroDivisionError:
            odds_ratio, p = np.nan, np.nan
            ci_lower, ci_upper = np.nan, np.nan

        odds_ratios.append(odds_ratio)
        p_values.append(p)
        ci_lowers.append(ci_lower)
        ci_uppers.append(ci_upper)

    data['OR (Continuity Corrected)'] = odds_ratios
    data['P-value (Continuity Corrected)'] = p_values
    data['OR (Continuity Corrected) 95% CI Lower'] = ci_lowers
    data['OR (Continuity Corrected) 95% CI Upper'] = ci_uppers
    return data


# Example usage:
# corrected_OR_df = calculate_odds_ratio_ratio_correction(ordered_merged_df, cohort_name)
# non_corrected_OR_df = calculate_odds_ratio_no_correction(corrected_OR_df, cohort_name)

# Now both corrected and non-corrected dataframes will have OR, P-value, and 95% CI columns.


In [117]:
# Calculate corrected and non-corrected ORs and P-values
corrected_OR_df = calculate_odds_ratio_ratio_correction(ordered_merged_df, cohort_name)
non_corrected_OR_df = calculate_odds_ratio_no_correction(corrected_OR_df, cohort_name)

# Insert corrected and non-corrected columns after 'MAF (Control)'
columns_to_reorder = (
    list(non_corrected_OR_df.columns[:14]) +  # Columns up to 'MAF (Control)'
    [
    'OR (Continuity Corrected)', 'P-value (Continuity Corrected)',
    'OR (Continuity Corrected) 95% CI Lower', 'OR (Continuity Corrected) 95% CI Upper',
    'OR', 'P-value',
    'OR 95% CI Lower', 'OR 95% CI Upper'
    ] +
    list(non_corrected_OR_df.columns[14 :-8])  # Remaining columns
)

# Reorder the DataFrame
ordered_OR_df = non_corrected_OR_df[columns_to_reorder]



<ipython-input-116-c0a776a0c6f0>:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['OR (Continuity Corrected)'] = odds_ratios
<ipython-input-116-c0a776a0c6f0>:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['P-value (Continuity Corrected)'] = p_values
<ipython-input-116-c0a776a0c6f0>:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [118]:
ordered_OR_df.iloc[:, 6:7].join(ordered_OR_df.iloc[:, 10:22])

,Amino Acid Change,N (Case),N (Control),MAF (Case),MAF (Control),OR (Continuity Corrected),P-value (Continuity Corrected),OR (Continuity Corrected) 95% CI Lower,OR (Continuity Corrected) 95% CI Upper,OR,P-value,OR 95% CI Lower,OR 95% CI Upper
0,p.N59K,1,0,0.000402,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
1,p.A75A,0,1,0.000000,0.000906,0.148755,0.243500,0.006050,3.657444,NaN,NaN,NaN,NaN
5,p.H115P,2,0,0.000797,0.000000,2.238896,0.603078,0.107306,46.713729,NaN,NaN,NaN,NaN
6,p.L119P,2,2,0.000797,0.001776,0.446178,0.367676,0.077077,2.582818,0.445957,0.419962,0.062657,3.174052
7,p.L122L,2,1,0.000797,0.000888,0.744965,0.775856,0.098160,5.653742,0.893515,0.926816,0.080849,9.874753
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,p.T2356I,2,0,0.000797,0.000000,2.238896,0.603078,0.107306,46.713729,NaN,NaN,NaN,NaN
115,p.L2363L,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
116,p.G2385R,0,2,0.000000,0.001776,0.089093,0.118756,0.004270,1.858900,NaN,NaN,NaN,NaN
118,p.R2394R,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN


In [119]:
ordered_OR_df.iloc[34:52, 6:7].join(ordered_OR_df.iloc[34:52, 10:22])

,Amino Acid Change,N (Case),N (Control),MAF (Case),MAF (Control),OR (Continuity Corrected),P-value (Continuity Corrected),OR (Continuity Corrected) 95% CI Lower,OR (Continuity Corrected) 95% CI Upper,OR,P-value,OR 95% CI Lower,OR 95% CI Upper
69,p.L1590V,0,1,0.000000,0.000888,0.148755,0.243500,0.006050,3.657444,NaN,NaN,NaN,NaN
73,p.K1620E,0,1,0.000000,0.000888,0.148755,0.243500,0.006050,3.657444,NaN,NaN,NaN,NaN
74,p.K1620R,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
75,p.H1621R,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
76,p.P1622H,0,1,0.000000,0.000888,0.148755,0.243500,0.006050,3.657444,NaN,NaN,NaN,NaN
77,p.R1628P,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
78,p.P1642P,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
79,p.L1653L,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
80,p.L1694L,1,0,0.000398,0.000000,1.342263,0.857020,0.054592,33.002252,NaN,NaN,NaN,NaN
82,p.S1721S,2,0,0.000797,0.000000,2.238896,0.603078,0.107306,46.713729,NaN,NaN,NaN,NaN


In [120]:
# save into excel file
#ordered_merged_df.to_excel(f'{directory}/{cohort_name}_annotated.xlsx', index=False)
ordered_OR_df.to_excel(f'{directory}/{cohort_name}_annotated_OR_CI_new_p-value_OR.xlsx', index=False)